## 1. Importação e instalação das dependê

In [ ]:
#pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe==0.8.6.2 sklearn matplotlib --user

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## 2. Mapeamento dos pontos-chave utilizando MediaPipe Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Modelo holístico
mp_drawing = mp.solutions.drawing_utils # Desenho dos pontos

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # CONVERSÃO DAS CORES BGR para RGB
    image.flags.writeable = False                  # Imagem não é mais alterável
    results = model.process(image)                 # Faz a predição
    image.flags.writeable = True                   # Imagem volta a ser alterável 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # CONVERSÃO DAS CORES RGB para BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Desenha conexões do rosto
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Desenha conexões da pose
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Desenha conexões da mão esquerda
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Desenha conexões da mão direita

In [5]:
def draw_styled_landmarks(image, results):
    # Desenha conexões do rosto
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Desenha conexões da pose
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Desenha conexões da mão esquerda
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Desenha conexões da mão direita  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

## 3. Extrair os ponto-chave

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

## 4. Configuração das pastas para dados

In [7]:
# Caminho para exportar o dado dos gestos
DATA_PATH = os.path.join('MP_Data') 

# Acoes que iremos detectar
actions = np.array(['lingua_de_sinais', 'surdo', 'conhecer', 'amigo','entender','denovo','estudar',])

# 65 videos para cada pose
no_sequences = 50

# Cada possuira 30 frames
sequence_length = 30

In [8]:
for action in actions: # para cada gesto
    for sequence in range(no_sequences): # para cada frame
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence))) # cria o diretório
        except:
            pass

## 5. Coleta os ponto-chave para treinamento e testes

In [ ]:
cap = cv2.VideoCapture(0)
# Configura o modelo mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop pelos gestos
    for action in actions:
        # Loop pelas sequências (videos)
        for sequence in range(no_sequences):
            # Loop pelos frames
            for frame_num in range(sequence_length):

                # Le a camera
                ret, frame = cap.read()

                # Faz as detecções
                image, results = mediapipe_detection(frame, holistic)
                # print(results)

                # Desenha os ponto-chave
                draw_styled_landmarks(image, results)
                
                # se for o início de um vídeo
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Mostra na tela
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Mostra na tela
                    cv2.imshow('OpenCV Feed', image)
                
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Sai do programa
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

## 6. Configuração e treinamento da Rede Neural LSTM

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
# Coleta os dados das pastas
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
X = np.array(sequences) # dados de treinamento

In [14]:
y = to_categorical(labels).astype(int) # rótulo dos sinais

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # separação dos dados de treinamento, 20% será utilizado para testes

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [19]:
#from tensorflow.keras.callbacks import EarlyStopping
#early_stop = EarlyStopping(monitor='loss',mode='min',verbose=1,patience=25)
model.fit(X_train, y_train, epochs=120, callbacks=[tb_callback])

Epoch 1/120
10/10 [==============================] - 4s 97ms/step - loss: 2.8674 - categorical_accuracy: 0.1574
Epoch 2/120
10/10 [==============================] - 1s 55ms/step - loss: 1.9034 - categorical_accuracy: 0.1482
Epoch 3/120
10/10 [==============================] - 1s 54ms/step - loss: 1.6725 - categorical_accuracy: 0.2309
Epoch 4/120
10/10 [==============================] - 1s 52ms/step - loss: 1.6071 - categorical_accuracy: 0.3402
Epoch 5/120
10/10 [==============================] - 1s 52ms/step - loss: 1.6006 - categorical_accuracy: 0.3848
Epoch 6/120
10/10 [==============================] - 1s 53ms/step - loss: 1.5770 - categorical_accuracy: 0.4116
Epoch 7/120
10/10 [==============================] - 1s 52ms/step - loss: 1.3970 - categorical_accuracy: 0.4350
Epoch 8/120
10/10 [==============================] - 1s 52ms/step - loss: 1.4054 - categorical_accuracy: 0.4190
Epoch 9/120
10/10 [==============================] - 1s 54ms/step - loss: 1.1650 - categorical_accuracy:

10/10 [==============================] - 1s 54ms/step - loss: 1.0707 - categorical_accuracy: 0.5808
Epoch 74/120
10/10 [==============================] - 1s 53ms/step - loss: 0.8287 - categorical_accuracy: 0.6822
Epoch 75/120
10/10 [==============================] - 1s 51ms/step - loss: 0.5899 - categorical_accuracy: 0.7325
Epoch 76/120
10/10 [==============================] - 1s 53ms/step - loss: 0.3979 - categorical_accuracy: 0.8843
Epoch 77/120
10/10 [==============================] - 1s 51ms/step - loss: 0.3089 - categorical_accuracy: 0.9045
Epoch 78/120
10/10 [==============================] - 1s 52ms/step - loss: 0.6483 - categorical_accuracy: 0.7983
Epoch 79/120
10/10 [==============================] - 1s 54ms/step - loss: 0.6885 - categorical_accuracy: 0.8065
Epoch 80/120
10/10 [==============================] - 1s 54ms/step - loss: 0.4805 - categorical_accuracy: 0.8196
Epoch 81/120
10/10 [==============================] - 1s 54ms/step - loss: 0.3186 - categorical_accuracy: 0.8

KeyboardInterrupt: 

In [ ]:
model.summary()

## 7. Salva o modelo

In [ ]:
model.save('action_v6.h5') # salva o modelo

In [ ]:
model.load_weights('action_v5.h5') # carrega o modelo

In [ ]:
model.evaluate(X_test,y_test) # teste da acurácia

## 8. Teste em tempo real

In [ ]:
from scipy import stats

In [ ]:
# função que exibe a porcentagem de cada gesto
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence = []
sentence = []
predictions = []
threshold = 0.5 # tempo de espera para confirmar o sinal

cap = cv2.VideoCapture(0)
# configura o modelo mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # le a camera
        ret, frame = cap.read()

        # faz as detecções
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # desenha os ponto-chave
        draw_styled_landmarks(image, results)
        
        # lógica da predição
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            # lógica da visualização
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # vizualicação das probabilidades
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # mostra na tela
        cv2.imshow('OpenCV Feed', image)

        # sai do programa
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()